In [3]:
# a fast class for polynomial arithmetic in sage
# the idea is to keep working in the ring ZZ[x] and implement the reduction
# mod x^N+1 only when necessary, and in a fast manner.
# moreover we want to keep track of the coefficient modulus, so that we only reduce
# when necessary.

import time

class P:
    @classmethod
    def setparameters(cls, N):
        cls.N = N
        cls.R = PolynomialRing(ZZ, 'x')
        cls.x = cls.R.gen()
        cls.quo = cls.x^N + 1

    def __init__(self, coeffs, modulus=None):
        self.mod = modulus if modulus else 0
        try:
            if coeffs.parent() == self.R:
                self.v = coeffs
        except:
            assert len(coeffs) <= self.N, f"polynomial degree {len(coeffs)} exceeds N={self.N}"
            self.v = self.R(coeffs)

    def __repr__(self):
        if self.mod != 0:
            reduced = self.reduce(self.v, self.mod)
            central = [i if i <= self.mod//2 else i-self.mod for i in reduced.list()]
            return repr(self.R(central))
        return repr(self.v)

    def reduce(self, coeffs, modulus):
        return coeffs.change_ring(Zmod(modulus)).change_ring(ZZ)

    def newmodulus(self, a, b):
        if a.mod == 0 or b.mod == 0:
            return max(a.mod, b.mod)
        elif a.mod == b.mod:
            return a.mod
        else:
            raise "Nonzero moduli must be equal"

    def makepoly(self, a):
        # make a class P element from a
        return a if isinstance(a, P) else P(a, modulus=a[0].modulus())

    def reducepoly(self):
        return self.v.truncate(self.N)-self.v.shift(-self.N)

    def __mod__(self, other):
        if isinstance(other, Integer) or isinstance(other, int):
            reduced = self.reduce(self.v, other)
            return P(reduced, modulus=other)
        else:
            raise NotImplementedError
        
    def __add__(self, other):
        other = self.makepoly(other)
        mod = self.newmodulus(self, other)
        return P(self.v + other.v, modulus=mod)

    def __iadd__(self, other): # overwriting
        self.v += self.makepoly(other).v
        return self
    
    def __sub__(self, other):
        other = self.makepoly(other)
        mod = self.newmodulus(self, other)
        return P(self.v - other.v, modulus=mod)

    def __isub__(self, other):
        self.v -= self.makepoly(other).v
        return self

    def __neg__(self):
        return P(-self.v, modulus=self.mod)

    def __mul__(self, other):
        if isinstance(other, P):
            result = self.v * other.v 
            temp = result.truncate(self.N)-result.shift(-self.N)
            return P(temp, modulus=self.newmodulus(self, other))
        # missing: multiplication by integer
        else:
            return P(self.v * other, modulus=self.newmodulus(self, other))
    
    def __imul__(self, other):
        if isinstance(other, P):
            self.v *= other.v
            self.v = self.v.truncate(self.N)-self.v.shift(-self.N)
        else:
            self.v *= other
        return self

    def __abs__(self):
        coeffs = self.reduce(self.v, self.mod)
        return max([abs(i) if i <= self.mod//2 else abs(i-self.mod) for i in coeffs.list()])
        


In [6]:
N = 2**12
q = 2**1000
iterations = 5
P.setparameters(N)
Rq = PolynomialRing(Zmod(q), 'x').quotient(x^N+1)
a = Rq.random_element()
b = Rq.random_element()
c = P(a.list(), modulus=q)
d = P(b.list(), modulus=q)
start = time.time()
for i in range(iterations):
    c = (c * d) % q
    # a = a * b
end = time.time()
print(f"Time for {iterations} multiplications: {end-start:.2f} seconds")

Time for 5 multiplications: 0.11 seconds
